In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import operator
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize, Bounds
from functools import partial

In [2]:
def cube(x):
    return x**3
def module(x):
    return abs(x-0.2)
def sinus(x):
    return x*math.sin(1/x)

In [3]:
def drawplot(function, methods, a,b, eps):
    for method in methods:
        attempts = method(function, a, b, eps)
        print("Attempts for function '{}' by method '{}' is {}".format(function.__name__, method.__name__, attempts))
    print("\n")

In [4]:
def exhaustiveSearchOD(function, a, b, eps):
    n = math.ceil((b-a)/eps)
    x = np.linspace(a,b,n)
    y = [function(i) for i in x]
    fig = plt.figure(figsize=(10,5))
    plt.style.use('seaborn-darkgrid')
    plt.plot(x, y, marker='', color='black', linewidth=0.4)
    
    xk = [a + k*(b-a)/n for k in range(n)]
    f_xk = {function(item):item for item in xk}
    MIN = None
    attempt = 0
    for key in f_xk:
        attempt += 1
        if MIN == None:
            MIN = key
        if key < MIN:
            MIN = key
        plt.scatter(f_xk[key],key,s=0.2,color="black")
            
    y_MIN = MIN
    x_MIN = f_xk[MIN]
    
    plt.plot(x_MIN, y_MIN, 'o', color="red")
    plt.title(function.__name__ + " exhaustiveSearchOD", fontsize=12, fontweight=0, color='black')
    plt.savefig("pictures/"+function.__name__ + "_exhaustiveSearchOD"+".png")
    plt.close(fig)
    return attempt

In [5]:
def golden(function, a, b, eps):
    n = math.ceil((b-a)/eps)
    x = np.linspace(a,b,n)
    y = [function(i) for i in x]
    fig = plt.figure(figsize=(10,5))
    plt.style.use('seaborn-darkgrid')
    plt.plot(x, y, marker='', color='black', linewidth=0.4, alpha=1)
    
    a0 = a
    b0 = b
    delta = 0.0001
    attm = 0
    while abs(a0-b0) >= eps:
        attm += 1
        x1 = a0+(b0-a0)*((3-math.sqrt(5))/2)
        x2 = b0+(b0-a0)*((math.sqrt(5)-3)/2)
        f_x1 = function(x1)
        f_x2 = function(x2)
        if f_x1 <= f_x2:
            b0 = x2
        else:
            a0 = x1
        x_MIN = (a0+b0)/2
        y_MIN = function(x_MIN)
        plt.scatter(x_MIN,y_MIN,s=3,color="black")
        
    x_MIN = (a0+b0)/2
    y_MIN = function(x_MIN)
    
    plt.plot(x_MIN, y_MIN, 'o', color="red")
    plt.title(function.__name__ + " golden", fontsize=12, fontweight=0, color='black')
    plt.savefig("pictures/"+function.__name__ + "_golden"+".png")
    plt.close(fig)
    return attm

In [6]:
def dichotomy(function, a, b, eps):
    n = math.ceil((b-a)/eps)
    x = np.linspace(a,b,n)
    y = [function(i) for i in x]
    fig = plt.figure(figsize=(10,5))
    plt.style.use('seaborn-darkgrid')
    plt.plot(x, y, marker='', color='black', linewidth=0.4, alpha=1)
    
    a0 = a
    b0 = b
    delta = 0.0001
    attm = 0
    while abs(a0-b0) >= eps:
        attm += 1
        x1 = (a0+b0-delta)/2
        x2 = (a0+b0+delta)/2
        f_x1 = function(x1)
        f_x2 = function(x2)
        if f_x1 <= f_x2:
            b0 = x2
        else:
            a0 = x1
        x_MIN = (a0+b0)/2
        y_MIN = function(x_MIN)
        plt.scatter(x_MIN,y_MIN,s=3,color="black")
        
    x_MIN = (a0+b0)/2
    y_MIN = function(x_MIN)
    
    plt.plot(x_MIN, y_MIN, 'o', color="red")
    plt.title(function.__name__ + " dichotomy", fontsize=12, fontweight=0, color='black')
    plt.savefig("pictures/"+function.__name__ + "_dichotomy"+".png")
    plt.close(fig)
    return attm

In [7]:
#Task 2.1
eps=0.001
tasks = [{"a" : 0, "b" : 1, "function" : cube},
        {"a" : 0, "b" : 1, "function" : module},
        {"a" : 0.01, "b" : 1, "function" : sinus}]
methods = [exhaustiveSearchOD, dichotomy,golden]
for task in tasks:
    drawplot(task["function"], methods, task["a"], task["b"], eps)

Attempts for function 'cube' by method 'exhaustiveSearchOD' is 1000
Attempts for function 'cube' by method 'dichotomy' is 11
Attempts for function 'cube' by method 'golden' is 15


Attempts for function 'module' by method 'exhaustiveSearchOD' is 932
Attempts for function 'module' by method 'dichotomy' is 11
Attempts for function 'module' by method 'golden' is 15


Attempts for function 'sinus' by method 'exhaustiveSearchOD' is 990
Attempts for function 'sinus' by method 'dichotomy' is 11
Attempts for function 'sinus' by method 'golden' is 15




In [206]:
#Task 2.2

In [8]:
def D_sum(a,b,x,y,function):
    f_res = [function(i,a,b) for i in x]
    result = sum([pow(f_i+y_i, 2) for f_i, y_i in zip(f_res,y)])
    return result

In [9]:
def exhaustiveSearchMD(d_func, function, x, y, name, a_vec, b_vec,az):
    result = {}
    for i in a_vec:
        for j in b_vec:
            result[(i,j)] = d_func(i,j,x,y,function)
    X_MIN, Y_MIN = min(result.items(), key=operator.itemgetter(1))[0]
    
    point = {(X_MIN, Y_MIN) : result[(X_MIN, Y_MIN)]}
    
    X, Y = np.meshgrid(a_vec, b_vec)
    Z = d_func(X,Y,x,y,function)
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection="3d")
    ax.plot_surface(X,Y,Z, rstride=5, cstride=5, cmap='winter', alpha=0.6)
    ax.set_xlabel('x',fontsize=18)
    ax.set_ylabel('y',fontsize=18)
    ax.set_zlabel('z',fontsize=18)
    ax.view_init(azim=az)
    ax.scatter(X_MIN, Y_MIN, result[(X_MIN, Y_MIN)], 'o', color="red")
    plt.title("Min by exhaustiveSearchMD for {} is: {}".format(name, point), fontsize=12, fontweight=0, color='black')
    plt.savefig("pictures/"+name + "_exhaustiveSearchMD"+".png")
    plt.close(fig)

    print("ExhaustiveSearchMD method for {} have {} steps".format(name, len(result)))

In [10]:
def gauss(d_func, function, x, y, name, eps, a_vec, b_vec,az):
    x_0 = random.random()
    y_0 = random.random()
    z_0 = d_func(x_0,y_0,x,y,function)
    
    points = [(x_0, y_0, z_0)]
    
    x_tmp = x_0
    res = {y_i : d_func(x_0,y_i,x,y,function) for y_i in b_vec}
    y_tmp = min(res.items(), key=operator.itemgetter(1))[0]
    z_tmp = res[y_tmp]
    
    flag = True
    while abs(points[-1][0] - x_tmp) > eps or abs(points[-1][1] - y_tmp) > eps:
        points.append((x_tmp, y_tmp, z_tmp))
        if flag:
            y_tmp = points[-1][1]
            res = {x_i : d_func(x_i,y_tmp,x,y,function) for x_i in a_vec}
            x_tmp = min(res.items(), key=operator.itemgetter(1))[0]
            z_tmp = res[x_tmp]
        else:
            x_tmp = points[-1][0]
            res = {y_i : d_func(x_tmp,y_i,x,y,function) for y_i in b_vec}
            y_tmp = min(res.items(), key=operator.itemgetter(1))[0]
            z_tmp = res[y_tmp]
        flag = not flag
        
        
    X, Y = np.meshgrid(a_vec, b_vec)
    Z = d_func(X,Y,x,y,function)
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection="3d")
    ax.plot_surface(X,Y,Z, rstride=5, cstride=5, cmap='winter',alpha=0.6)
    ax.set_xlabel('x',fontsize=18)
    ax.set_ylabel('y',fontsize=18)
    ax.set_zlabel('z',fontsize=18)
    ax.view_init(azim=az)
    plt.title("Min by gauss method for {} is: {}".format(name, {(points[-1][0], points[-1][1]) : points[-1][2]}), fontsize=12, fontweight=0, color='black')
    
    for item in points[:-1]:
        ax.scatter(item[0], item[1], item[2], 'o', color="black",s=30)
    
    ax.scatter(points[-1][0], points[-1][1], points[-1][2], 'o', color="red",s=30)
        
    plt.savefig("pictures/"+name + "_gauss"+".png")
    plt.close(fig)
    print("Gauss method for {} have {} steps".format(name, len(points)))

In [14]:
def nelder_mead(d_func, function, x, y, name, eps, a_vec, b_vec,az):
    X, Y = np.meshgrid(a_vec, b_vec)
    Z = d_func(X,Y,x,y,function)
    
    def func(c):
        return d_func(c[0],c[1],x,y,function)
    
    start_point = np.random.random_sample(2)
    res = minimize(func, start_point, method='nelder-mead', options={'xatol': eps, 'disp': False, 'adaptive': True})
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection="3d")
    ax.plot_surface(X,Y,Z, rstride=5, cstride=5, cmap='winter',alpha=0.6)
    ax.set_xlabel('x',fontsize=18)
    ax.set_ylabel('y',fontsize=18)
    ax.set_zlabel('z',fontsize=18)
    ax.view_init(azim=az)
    plt.title("Min by nelder_mead method for {} is: {}".format(name, {(res["x"][0], res["x"][1]) : res["fun"]}), fontsize=12, fontweight=0, color='black')
    
    ax.scatter(res["x"][0], res["x"][1], res["fun"], 'o', color="red",s=30)
    plt.savefig("pictures/"+name + "_nelder_mead"+".png")
    plt.close(fig)
    print("Nelder-Mead method for {} have {} steps".format(name, res["nit"]))

In [15]:
#Main of Task 2.2
k = 100
mu = 0
sigma = 1
eps = 0.001
alpha = random.randint(0,1)
beta = random.randint(0,1)

x_vec = np.linspace(0, k, k+1)
s = np.random.normal(mu, sigma, 1000)
y_vec = [alpha*x_i + beta + s_i for (x_i, s_i) in zip(x_vec,s)]

f_linear = lambda x,a,b: a*x + b
f_rational = lambda x,a,b: a/(1+b*x)

n = len(x_vec)
a_vec = np.linspace(-50, 50, n)
b_vec = np.linspace(-4, 10, n)

exhaustiveSearchMD(D_sum, f_linear, x_vec, y_vec, "f_linear", a_vec, b_vec,105)
exhaustiveSearchMD(D_sum, f_rational, x_vec, y_vec, "f_rational", a_vec, b_vec,130)
gauss(D_sum, f_linear, x_vec, y_vec, "f_linear", eps, a_vec, b_vec,105)
gauss(D_sum, f_rational, x_vec, y_vec, "f_rational", eps, a_vec, b_vec,130)
nelder_mead(D_sum, f_linear, x_vec, y_vec, "f_linear", eps, a_vec, b_vec,105)
nelder_mead(D_sum, f_rational, x_vec, y_vec, "f_rational", eps, a_vec, b_vec,130)

ExhaustiveSearchMD method for f_linear have 10201 steps
ExhaustiveSearchMD method for f_rational have 10201 steps
Gauss method for f_linear have 4 steps
Gauss method for f_rational have 3 steps
Nelder-Mead method for f_linear have 50 steps
Nelder-Mead method for f_rational have 52 steps
